In [ ]:
class Account:
    def __init__(self, account_id,balance, interest = 0):
        if not isinstance(account_id,str):
            raise TypeError('Account ID must be string')    
        for id in account_id:
            if id not in str(list(range(0,10))):
                raise ValueError('Account ID must be within 0-9')
        if not isinstance(balance,(int,float)):
            raise TypeError('Balance must be int or float') 
        self.account_id = account_id
        self.balance = balance
        self.interest = interest
    def __str__(self):
        return (f'{self.account_id},{self.balance},{self.interest}')

class Checking(Account):
    def __init__(self,account_id,balance,interest=0):
        super().__init__(account_id,balance,interest)
        self.interest = 0

class Savings(Account):
    def __init__(self,account_id,balance,interest=0):
        super().__init__(account_id,balance,interest)
        self.interest = 0